# 1. Connecting to and Querying a DataSource (XLSX - SQLLite)

This is a series of example of how you can connect Agents to different sources of data with Microsoft Azure's OpenAI Service

This is the 1st file of 3 similar scripts

## Note Upfront
If you want to your LLM model to have knowledge of your own data you can use  
  - Fine Tune your model with your data but this is very expensive to do (CPU) and doesn't work well with data that changes.  
  - RAG with a vectordatabase: this is suitable for documents, pictures, ..., but what about data in structured storage like databases or spreadsheets?
  - use a connector to a database: this is what this module is about
  
- In this series we will show how you can set up an Agent to 
  - use Langchain agents to connect to SQL Database or CSV file
  - via Azure OpenAI Assistants API (function calling + code interpreter) (stateful management + short term memory)
  - via Azure OpenAI Fucntion Calling: to perform tasks based on your questions  
- the 2nd part of this exercise connects to a SQLlite database, you have to have it installed on your environment  
- This demo is based upon on https://learn.deeplearning.ai/courses/building-your-own-database-agent  

## prereqs 
0. setup your local repo with a clone from this gitrepo. Don't forget to run the requirements.txt
1. have a MS Azure Account; with a valid subscription  
    - running through the course steps cost me < €0.50 but keep an eye on the costs. (portal.azure.com > search for 'Invoices' > Select 'Invoices' > Cost Management > Cost Analysis)  
    - remove the project when no longer needed to avoid recurrent costs.      
2. have a AI Foundry project with a deployed model
* Create a project -> Azure AI Foundry Resource  
    - chose a meaningful name, subscription you have setup, resource group (or create a new one), region (I typically pick Sweden Central as most of the AI Models are there)
* Pick the right urls & credentials !!  
    - pick the API Key and put it in your local .env with   
    - libraries: PICK AZURE OpenAI: something like https://<project_name>-resource.openai.azure.com/  

Your .env needs to look something like
AZURE_OPENAI_API_KEY=<your_api_key>  
AZURE_URL=https://<project_name>-resource.openai.azure.com/<br>

* Deploy a model: You can pick anymodel but I work with the gpt-4.1-mini model and model version. Put that in the .env file to have all parameters in one location  
AZURE_OPENAI_MODEL=gpt-4.1-mini  
AZURE_OPENAI_MODEL_VERSION=2025-03-01-preview

## 1.1 Connecting to XLSX   
In this example we will have a xls and create a Langchain agent that   
- loads the excel in a panda  
- understands the excel file   
- is able to translate natural language, logical questions into queries  

### 1.1.1 Step 1: Setting up your Azure & Langchain

In [1]:
import os
import pandas as pd
from IPython.display import Markdown, HTML, display
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(override=True)   # avoid the sytem set parameters to override your local the .env file

True

In [2]:

from openai import AzureOpenAI

# Azure OpenAI Configuration (CORRECT endpoint from Azure Portal)
# endpoint = "https://js-alphacentauri-resource.cognitiveservices.azure.com/"
endpoint = os.getenv("AZURE_URL")
deployment = os.getenv("AZURE_OPENAI_MODEL")
v_model = os.getenv("AZURE_OPENAI_MODEL")
api_version = os.getenv("AZURE_OPENAI_MODEL_VERSION")  # Updated to latest API version for Responses API support

# Get API key from environment
subscription_key = os.getenv("AZURE_OPENAI_API_KEY")

# if issues, uncomment the following to validate the keys are correctly read
# print("✅ Azure OpenAI client configured")
# print(f"Endpoint: {endpoint}")
# print(f"Deployment: {deployment}")
# print(f"API Version: {api_version}")    
# print(f"Subscription Key: {subscription_key}")
# print(f"API Key (1st 5 Chars): {subscription_key[:5]}...")

# Create Azure OpenAI client
client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

In [3]:
# Test the connection
response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "Say hello and tell me you're working! But to lighten up the atmosphere, tell a joke about Generative AI and Langchain.",
        }
    ],
    model=deployment
)

print(response.choices[0].message.content)


Hello! I'm working hard and ready to assist you. To lighten up the mood, here's a joke for you:

Why did the Generative AI break up with Langchain?  
Because it couldn't handle all the *chains* of commitment! 😄


### LangChain Integration


In [4]:
from langchain_openai import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

v_messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "Translate this sentence from English "
    "to French, West Vlaams and Spanish. I like football, bananas and pink panthers.",
        }
    ]

# Create LangChain model
model = AzureChatOpenAI(
    openai_api_version=api_version,
    azure_deployment=deployment,
    azure_endpoint=endpoint,
    api_key=subscription_key
)

response = model.invoke(v_messages)
print(response.content)


Sure! Here is the translation of the sentence "I like football, bananas and pink panthers." in French, West Vlaams, and Spanish:

French:  
J'aime le football, les bananes et les panthères roses.

West Vlaams:  
Ik vin voetbal, bananen en roze panters plezant.

Spanish:  
Me gustan el fútbol, los plátanos y las panteras rosas.


### Streaming Response Example


In [5]:


# Stream the output for better user experience
response = client.chat.completions.create(
    stream=True,
    messages=v_messages,
    model=deployment,
)

print("Streaming response:")
for update in response:
    if update.choices:
        print(update.choices[0].delta.content or "", end="")


Streaming response:
Sure! Here is the translation of the sentence "I like football, bananas and pink panthers." into French, West Vlaams, and Spanish:

- French: J'aime le football, les bananes et les panthères roses.
- West Vlaams: Ik vin voetbal, bananen en roze panters plezant.
- Spanish: Me gustan el fútbol, los plátanos y las panteras rosas.

### 🔍 Debug: Verify Deployment Name


In [6]:

output_message = model.invoke((v_messages))
print(output_message.model_dump_json(indent=2))

{
  "content": "Sure! Here is the translation of the sentence \"I like football, bananas and pink panthers.\" in French, West Flemish, and Spanish:\n\nFrench: J'aime le football, les bananes et les panthères roses.  \nWest Flemish: Ik zin goand op voetbal, bananen en roze panters.  \nSpanish: Me gustan el fútbol, los plátanos y las panteras rosas.",
  "additional_kwargs": {
    "refusal": null
  },
  "response_metadata": {
    "token_usage": {
      "completion_tokens": 85,
      "prompt_tokens": 41,
      "total_tokens": 126,
      "completion_tokens_details": {
        "accepted_prediction_tokens": 0,
        "audio_tokens": 0,
        "reasoning_tokens": 0,
        "rejected_prediction_tokens": 0
      },
      "prompt_tokens_details": {
        "audio_tokens": 0,
        "cached_tokens": 0
      }
    },
    "model_name": "gpt-4.1-mini-2025-04-14",
    "system_fingerprint": "fp_3dcd5944f5",
    "id": "chatcmpl-CXmLvf4GUunwHGtwXPWGpLvJj1fk7",
    "service_tier": null,
    "prompt_fi

### 1.1.2 Step 2: Interacting with CSV Data

In [7]:
# Create LangChain model

# Already created above, but for your memory

# model = AzureChatOpenAI(
#     openai_api_version=api_version,
#     azure_deployment=deployment,
#     azure_endpoint=endpoint,
#     api_key=subscription_key
# )


#### Load the dataset
- load the dataset from the csv file (demo file is seperated with #)


In [8]:
#read in the data from the csv file
df = pd.read_csv("./data/synthetic_sales_data.csv", sep="#").fillna(value=0)

In [9]:
# Check if the file is read and display the first few rows with headers
print("First 5 rows of the dataset:")
print("=" * 80)
display(df.head())

# Alternative: use this if display() doesn't work
# print(df.head().to_string())

First 5 rows of the dataset:


,Region,ProductLine,Month,Revenue,UnitsSold,DiscountPercent,Cost,Profit,ProfitMargin,KPI_01,...,KPI_31,KPI_32,KPI_33,KPI_34,KPI_35,KPI_36,KPI_37,KPI_38,KPI_39,KPI_40
0,West,Electronics,2023-01-01,460182,1586,0.19,378293.21,81888.79,0.18,78.98,...,85.24,88.40,87.21,82.63,58.26,116.10,112.38,80.45,63.69,91.99
1,South,Electronics,2023-02-01,272420,716,0.17,169550.94,102869.06,0.38,72.87,...,119.08,63.75,139.68,119.74,116.19,134.28,128.87,136.67,66.65,87.59
2,North,Accessories,2023-03-01,321483,395,0.28,236719.17,84763.83,0.26,75.19,...,117.72,116.69,125.41,138.13,126.55,58.08,137.69,102.62,79.66,94.06
3,East,Electronics,2023-04-01,83002,662,0.04,61797.97,21204.03,0.26,80.44,...,73.80,93.08,61.00,125.94,80.71,119.66,92.01,145.04,76.08,121.29
4,South,Accessories,2023-05-01,175055,221,0.07,126326.89,48728.11,0.28,144.07,...,98.28,74.81,133.54,79.72,109.43,124.54,78.81,64.79,100.61,133.85


In [10]:
# Show detailed information about the dataset structure
print("📊 Dataset Information:")
print("-" * 80)
print(f"Total Rows: {len(df)}")
print(f"Total Columns: {len(df.columns)}")
print("\nColumn Names and Data Types:")
print("-" * 80)
for i, (col, dtype) in enumerate(zip(df.columns, df.dtypes), 1):
    print(f"{i:2d}. {col:30s} | Type: {dtype}")
print("-" * 80)


📊 Dataset Information:
--------------------------------------------------------------------------------
Total Rows: 100
Total Columns: 49

Column Names and Data Types:
--------------------------------------------------------------------------------
 1. Region                         | Type: object
 2. ProductLine                    | Type: object
 3. Month                          | Type: object
 4. Revenue                        | Type: int64
 5. UnitsSold                      | Type: int64
 6. DiscountPercent                | Type: float64
 7. Cost                           | Type: float64
 8. Profit                         | Type: float64
 9. ProfitMargin                   | Type: float64
10. KPI_01                         | Type: float64
11. KPI_02                         | Type: float64
12. KPI_03                         | Type: float64
13. KPI_04                         | Type: float64
14. KPI_05                         | Type: float64
15. KPI_06                         | Type: f

#### <span style="color: red">SOME ATTENTION REQUIRED</span> 
create_pandas_dataframe_agent() makes an agent that:
- looks at your datframe (df) and understands it 
- interpretes and understands your prompt (“How many rows are there?”)
- write itself Python-code to execute your command
- and executes your code in a “Python REPL” (read–eval–print-loop) inside your process.
The “REPL” is a mini Python console inside your process.
The LLM or a malicious prompt could execute any type of code in the REPL, not only secure code but also delete files
By default you can't run this, unless a parameter is set.
--> We have controle here over the data --> **allow_dangerous_code=True** added

Do Not use this in production!


In [11]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(
    llm=model,
    df=df,
    verbose=True, 
    allow_dangerous_code=True,
    max_iterations=30,  # Increase from default 15
    max_execution_time=300  # 5 minutes timeout
)

agent.invoke("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: To find out how many rows are there in the dataframe `df`, I need to check the shape of the dataframe and look at the number of rows.

Action: python_repl_ast
Action Input: df.shape(100, 49)Thought: The shape of the dataframe is (100, 49), which means there are 100 rows and 49 columns.

Final Answer: There are 100 rows in the dataframe.

> Finished chain.


{'input': 'how many rows are there?',
 'output': 'There are 100 rows in the dataframe.'}

In [ ]:
# agent.invoke("What can you tell me about the data?")
# agent.invoke("List me all the diffrent productlines and their individual summed uprevenue")
agent.invoke("List per region which productline sold most units in total and how many units were sold.")

# validate the data by opening the xlsx in the data directory, and create pivot tables to proof the point.
# Final Answer: 
# - East: Clothing, 9992 units sold
# - North: Furniture, 9256 units sold
# - South: Furniture, 9829 units sold
# - West: Furniture, 8620 units sold




> Entering new AgentExecutor chain...
Thought: I need to group the dataframe by 'Region' and 'ProductLine', sum the 'UnitsSold' for each group, then find the product line with the maximum units sold for each region.

Action: [python_repl_ast]
Action Input: df.groupby(['Region', 'ProductLine'])['UnitsSold'].sum().reset_index().sort_values(['Region', 'UnitsSold'], ascending=[True, False])[python_repl_ast] is not a valid tool, try one of [python_repl_ast].Thought: I had a typo in using the tool name, I will retry the grouping and sorting using the correct tool name.

Action: python_repl_ast
Action Input: df.groupby(['Region', 'ProductLine'])['UnitsSold'].sum().reset_index().sort_values(['Region', 'UnitsSold'], ascending=[True, False])   Region  ProductLine  UnitsSold
1    East     Clothing       9992
0    East  Accessories       7715
3    East    Furniture       4558
2    East  Electronics       4076
7   North    Furniture       9256
5   North     Clothing       9161
6   North  Electron

{'input': 'List per region which productline sold most units in total and how many units were sold.',
 'output': '- East: Clothing, 9992 units sold\n- North: Furniture, 9256 units sold\n- South: Furniture, 9829 units sold\n- West: Furniture, 8620 units sold'}

In [16]:
CSV_PROMPT_PREFIX = """
First set the pandas display options to show all the columns,
get the column names, then answer the question.
"""

CSV_PROMPT_SUFFIX = """
- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.
- **ALWAYS**, as part of your "Final Answer", explain how you got
to the answer on a section that starts with: "\n\nExplanation:\n".
In the explanation, mention the column names that you used to get
to the final answer.
"""

#QUESTION = "How may patients were hospitalized during July 2020" 
#"in Texas, and nationwide as the total of all states?"
SIMPLE_PROMPT_SALES1 = """
Sum per region the UnitsSold per region for the ProductLine = Electronics where KPI_01 is greater than 100. 

Instructions:
1. Filter the data for ProductLine = 'Electronics' and KPI_01 > 100
2. Sum per region the UnitsSold
3. Provide both answers with a clear explanation
"""

SIMPLE_PROMPT_SALES2 = """
How many toys units are sold in the month of July, regardless the year?
"""
output = agent.invoke(CSV_PROMPT_PREFIX + SIMPLE_PROMPT_SALES1 + CSV_PROMPT_SUFFIX)

## try out the SIMPLE_PROMPT_SALES2 



> Entering new AgentExecutor chain...
Question: 
First set the pandas display options to show all the columns,
get the column names, then answer the question.

Sum per region the UnitsSold per region for the ProductLine = Electronics where KPI_01 is greater than 100. 

Instructions:
1. Filter the data for ProductLine = 'Electronics' and KPI_01 > 100
2. Sum per region the UnitsSold
3. Provide both answers with a clear explanation

- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response using Markdown.
- **DO NOT MAKE UP A

| Region | UnitsSold |
|--------|-----------|
| East   | 3414      |
| North  | 1039      |
| South  | 4162      |
| West   | 932       |

In [ ]:
# Alternative: Use the complex prompt with reflection (now with increased iteration limit)
# Uncomment the line below to try the more thorough approach with validation
# agent.invoke(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)


In [17]:
# Access the output from the dictionary
print(output['output'])

# Or display it in markdown for better formatting
# display(Markdown(output['output']))

Here is the summary of UnitsSold per region for ProductLine = 'Electronics' where KPI_01 is greater than 100:

| Region | UnitsSold Sum |
|--------|--------------:|
| East   |          3414 |
| North  |          1039 |
| South  |          4162 |
| West   |           932 |

### Explanation:
To arrive at this result, I first set pandas display options to show all columns and get the column names. Then I filtered the dataframe `df` on two conditions:

- `ProductLine` must be `'Electronics'`
- `KPI_01` must be greater than 100

Using this filtered subset, I grouped the data by `Region` and summed up the column `UnitsSold` to get the total units sold per region meeting these criteria.

I validated the result by repeating the filtering and aggregation using a boolean mask method. Both approaches returned identical results, which confirms their correctness.

The key columns used are:
- `ProductLine`
- `KPI_01`
- `Region`
- `UnitsSold`

This systematic approach ensures the sum of units sold pe

## 1.2 Step 3: Connecting to a SQL Database

In [18]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase


In [19]:
import urllib.request

# Create data directory if it doesn't exist
os.makedirs("data", exist_ok=True)

# Download the file using Python (works on Windows, Mac, Linux)
# url = "https://covidtracking.com/data/download/all-states-history.csv"
# file_path = "./data/all-states-history.csv"

# print(f"Downloading from {url}...")
# urllib.request.urlretrieve(url, file_path)
# print(f"✅ File downloaded to {file_path}")

# # Load the data
# df = pd.read_csv(file_path).fillna(value=0)
# print(f"✅ Loaded {len(df)} rows of data")
df.head()

,Region,ProductLine,Month,Revenue,UnitsSold,DiscountPercent,Cost,Profit,ProfitMargin,KPI_01,KPI_02,KPI_03,KPI_04,KPI_05,KPI_06,KPI_07,KPI_08,KPI_09,KPI_10,KPI_11,KPI_12,KPI_13,KPI_14,KPI_15,KPI_16,KPI_17,KPI_18,KPI_19,KPI_20,KPI_21,KPI_22,KPI_23,KPI_24,KPI_25,KPI_26,KPI_27,KPI_28,KPI_29,KPI_30,KPI_31,KPI_32,KPI_33,KPI_34,KPI_35,KPI_36,KPI_37,KPI_38,KPI_39,KPI_40
0,West,Electronics,2023-01-01,460182,1586,0.19,378293.21,81888.79,0.18,78.98,131.43,67.89,61.48,140.88,132.24,116.88,110.36,104.15,73.07,135.75,90.99,58.48,144.70,54.94,66.14,56.83,136.04,108.21,92.36,140.81,103.30,132.46,99.79,98.26,135.57,117.97,69.34,139.70,146.40,85.24,88.40,87.21,82.63,58.26,116.10,112.38,80.45,63.69,91.99
1,South,Electronics,2023-02-01,272420,716,0.17,169550.94,102869.06,0.38,72.87,71.45,95.52,74.87,60.20,53.65,95.09,145.01,119.45,89.39,131.79,94.16,141.01,91.01,137.03,141.43,117.36,126.10,115.74,145.32,114.34,140.26,135.42,61.41,145.33,115.73,88.89,99.25,105.15,147.57,119.08,63.75,139.68,119.74,116.19,134.28,128.87,136.67,66.65,87.59
2,North,Accessories,2023-03-01,321483,395,0.28,236719.17,84763.83,0.26,75.19,114.02,137.72,149.00,64.70,88.70,79.65,54.90,88.22,120.71,104.80,106.09,80.42,55.21,140.75,100.66,118.43,55.39,75.67,105.04,99.97,74.59,120.91,62.54,138.35,125.01,110.46,113.42,79.43,120.59,117.72,116.69,125.41,138.13,126.55,58.08,137.69,102.62,79.66,94.06
3,East,Electronics,2023-04-01,83002,662,0.04,61797.97,21204.03,0.26,80.44,51.63,100.65,135.45,118.74,62.73,50.54,81.34,108.07,129.09,98.47,59.37,111.50,88.96,98.42,108.01,73.77,116.30,145.23,68.09,128.92,94.77,101.98,110.11,134.28,139.31,65.77,125.37,102.51,106.92,73.80,93.08,61.00,125.94,80.71,119.66,92.01,145.04,76.08,121.29
4,South,Accessories,2023-05-01,175055,221,0.07,126326.89,48728.11,0.28,144.07,115.16,137.46,89.70,73.55,128.74,81.67,145.80,123.11,109.19,69.63,81.22,139.17,59.29,124.11,142.05,90.83,117.76,58.40,130.62,147.39,77.71,113.48,75.07,126.50,142.83,54.45,93.60,51.80,85.67,98.28,74.81,133.54,79.72,109.43,124.54,78.81,64.79,100.61,133.85


In [ ]:
from sqlalchemy import create_engine

# Path to your SQLite database file
# database_file_path = "./data/test.db"
database_file_path = "./data/sales_db.db"

# Create an engine to connect to the SQLite database
# SQLite only requires the path to the database file
engine = create_engine(f'sqlite:///{database_file_path}')
# df = pd.read_csv(file_url).fillna(value = 0)
# df.to_sql(
#     'all_states_history',
#     con=engine,
#     if_exists='replace',
#     index=False
# )
df.to_sql(
    'sales_db',
    con=engine,
    if_exists='replace',
    index=False
)

100

#### Note
for more information about SQLAlchemy look at the included SQLAlchemy.ipynb

In [20]:
MSSQL_AGENT_PREFIX = """

You are an agent designed to interact with a SQL database.
## Instructions:
- Given an input question, create a syntactically correct {dialect} query
to run, then look at the results of the query and return the answer.
- Unless the user specifies a specific number of examples they wish to
obtain, **ALWAYS** limit your query to at most {top_k} results.
- You can order the results by a relevant column to return the most
interesting examples in the database.
- Never query for all the columns from a specific table, only ask for
the relevant columns given the question.
- You have access to tools for interacting with the database.
- You MUST double check your query before executing it.If you get an error
while executing a query,rewrite the query and try again.
- DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.)
to the database.
- DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS
OF THE CALCULATIONS YOU HAVE DONE.
- Your response should be in Markdown. However, **when running  a SQL Query
in "Action Input", do not include the markdown backticks**.
Those are only for formatting the response, not for executing the command.
- ALWAYS, as part of your final answer, explain how you got to the answer
on a section that starts with: "Explanation:". Include the SQL query as
part of the explanation section.
- If the question does not seem related to the database, just return
"I don\'t know" as the answer.
- Only use the below tools. Only use the information returned by the
below tools to construct your query and final answer.
- Do not make up table names, only use the tables returned by any of the
tools below.

## Tools:

"""

In [22]:
MSSQL_AGENT_FORMAT_INSTRUCTIONS = """

## Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do.
Action: the action to take, should be one of [{tool_names}].
Action Input: the input to the action.
Observation: the result of the action.
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer.
Final Answer: the final answer to the original input question.

Example of Final Answer:
<=== Beginning of example

Action: query_sql_db
Action Input: 
SELECT Revenue, productline, month
FROM sales_db
WHERE productline = 'Electronics'
order by Revenue desc
LIMIT 5

Observation:
[(460182,'Electronics','2023-01-01 00:00:00.000000',),
(452245,'Electronics','2027-10-01 00:00:00.000000',),
(395065,'Electronics','2025-10-01 00:00:00.000000',),
(336765,'Electronics','2031-02-01 00:00:00.000000',),
(278708,'Electronics','2024-08-01 00:00:00.000000',)]
Thought: Eureka, I now know the final answer
Final Answer: The top revenue for Electronics was 460182 in the month 2023-01-01 00:00:00.000000.

Explanation:
I queried the `sales_db` table for the `Revenue` column and 'Month' column where the productline
is 'Electronics'. The query returned a list of tuples
with the revenue and month for each Electronics soldanswer the question,
I took the first record of alle data in the list, which is 460182 revenue for month 2023-01-01 00:00:00.000000..
I used the following query

```sql
SELECT Revenue, productline, month
FROM sales_db
WHERE productline = 'Electronics'
order by Revenue desc
LIMIT 5
```
===> End of Example

"""

In [35]:
# Create LangChain model
model2 = AzureChatOpenAI(
    openai_api_version=api_version,
    azure_deployment=deployment,
    azure_endpoint=endpoint,
    api_key=subscription_key,
    temperature = 0,
    max_tokens=5000  # Increased to allow proper agent format responses with SQL queries and explanations
)
db = SQLDatabase.from_uri(f'sqlite:///{database_file_path}')
toolkit = SQLDatabaseToolkit(db=db, llm=model2)

In [ ]:
QUESTION = """How much money did we make and how units did we sell?
"""

QUESTION2 ="How much money per units sold did we make starting from 2025-01-01 onwards?"
# the answer to question 2 is 228,6833167789572 money per units sold starting from 2025-01-01

QUESTION3 ="""What is the average revenue per unit for all sales from Jan 2025 onwards? Return a single number."""
## the answer should be 228 and some change, but the LLM misses here as he is taking the average per month .... 

QUESTION3b ="""What is the average revenue per unit for all sales from Jan 2025 onwards? Return a single number.
Calculate first the sum of all revenue and the sum of all units sold for the period. Divide the sum of the revenue by the sum of all units sold for the period """"main copy.ipynb"
# the answer to question 3 is 228,6833167789572 money per units sold starting from 2025-01-01

## Important note
# This "How much money per units sold did we make starting from January 2025 onwards?" crashed the agent as the agent got confused with should i return a single number or a list of numbers
# of averages per month or a single average for all the months. 
# this combined with the fact the example format instructions only show simple single-value answers, not complex multi-row results

agent_executor_SQL = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    format_instructions = MSSQL_AGENT_FORMAT_INSTRUCTIONS,
    llm=model2,
    toolkit=toolkit,
    top_k=30,
    verbose=True,
    handle_parsing_errors=True   # don't crash on parsing errors
)

In [48]:
output_2 = agent_executor_SQL.invoke({"input": QUESTION3b})





> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: sales_dbAction: sql_db_schema
Action Input: sales_db
CREATE TABLE sales_db (
	"Region" TEXT, 
	"ProductLine" TEXT, 
	"Month" DATETIME, 
	"Revenue" BIGINT, 
	"UnitsSold" BIGINT, 
	"DiscountPercent" FLOAT, 
	"Cost" FLOAT, 
	"Profit" FLOAT, 
	"ProfitMargin" FLOAT, 
	"KPI_01" FLOAT, 
	"KPI_02" FLOAT, 
	"KPI_03" FLOAT, 
	"KPI_04" FLOAT, 
	"KPI_05" FLOAT, 
	"KPI_06" FLOAT, 
	"KPI_07" FLOAT, 
	"KPI_08" FLOAT, 
	"KPI_09" FLOAT, 
	"KPI_10" FLOAT, 
	"KPI_11" FLOAT, 
	"KPI_12" FLOAT, 
	"KPI_13" FLOAT, 
	"KPI_14" FLOAT, 
	"KPI_15" FLOAT, 
	"KPI_16" FLOAT, 
	"KPI_17" FLOAT, 
	"KPI_18" FLOAT, 
	"KPI_19" FLOAT, 
	"KPI_20" FLOAT, 
	"KPI_21" FLOAT, 
	"KPI_22" FLOAT, 
	"KPI_23" FLOAT, 
	"KPI_24" FLOAT, 
	"KPI_25" FLOAT, 
	"KPI_26" FLOAT, 
	"KPI_27" FLOAT, 
	"KPI_28" FLOAT, 
	"KPI_29" FLOAT, 
	"KPI_30" FLOAT, 
	"KPI_31" FLOAT, 
	"KPI_32" FLOAT, 
	"KPI_33" FLOAT, 
	"KPI_34" FLOAT, 
	"KPI_35" FLOAT, 
	"KPI_36" FLOAT, 
	"K

In [49]:

# Or display it in markdown for better formatting
display(Markdown(output_2['input']))
display(Markdown(output_2['output']))

What is the average revenue per unit for all sales from Jan 2025 onwards? Return a single number.
Calculate first the sum of all revenue and the sum of all units sold for the period. Divide the sum of the revenue by the sum of all units sold for the period main copy.ipynb

The average revenue per unit for all sales from January 2025 onwards is approximately 228.68.

Explanation:  
I queried the `sales_db` table to get the total revenue and total units sold from January 2025 onwards. Then, I divided the total revenue by the total units sold to get the average revenue per unit. The SQL query used was:

```sql
SELECT SUM(Revenue) as total_revenue, SUM(UnitsSold) as total_units
FROM sales_db
WHERE Month >= '2025-01-01'
LIMIT 30
```

The result was a total revenue of 19,109,464 and total units sold of 83,568. Dividing these gives an average revenue per unit of about 228.68.

In [59]:
QUESTION5 = """Which productline had the second average profit margin in the North region during 2025. And how much lower was the profit margin versus the highest average profit margin in the North region during 2025
"""

agent_executor_SQL = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    format_instructions = MSSQL_AGENT_FORMAT_INSTRUCTIONS,
    llm=model2,
    toolkit=toolkit,
    top_k=30,
    verbose=True
)

In [60]:
output_challenge = agent_executor_SQL.invoke({"input": QUESTION5})



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: 
""sales_dbAction: sql_db_schema
Action Input: sales_db
CREATE TABLE sales_db (
	"Region" TEXT, 
	"ProductLine" TEXT, 
	"Month" DATETIME, 
	"Revenue" BIGINT, 
	"UnitsSold" BIGINT, 
	"DiscountPercent" FLOAT, 
	"Cost" FLOAT, 
	"Profit" FLOAT, 
	"ProfitMargin" FLOAT, 
	"KPI_01" FLOAT, 
	"KPI_02" FLOAT, 
	"KPI_03" FLOAT, 
	"KPI_04" FLOAT, 
	"KPI_05" FLOAT, 
	"KPI_06" FLOAT, 
	"KPI_07" FLOAT, 
	"KPI_08" FLOAT, 
	"KPI_09" FLOAT, 
	"KPI_10" FLOAT, 
	"KPI_11" FLOAT, 
	"KPI_12" FLOAT, 
	"KPI_13" FLOAT, 
	"KPI_14" FLOAT, 
	"KPI_15" FLOAT, 
	"KPI_16" FLOAT, 
	"KPI_17" FLOAT, 
	"KPI_18" FLOAT, 
	"KPI_19" FLOAT, 
	"KPI_20" FLOAT, 
	"KPI_21" FLOAT, 
	"KPI_22" FLOAT, 
	"KPI_23" FLOAT, 
	"KPI_24" FLOAT, 
	"KPI_25" FLOAT, 
	"KPI_26" FLOAT, 
	"KPI_27" FLOAT, 
	"KPI_28" FLOAT, 
	"KPI_29" FLOAT, 
	"KPI_30" FLOAT, 
	"KPI_31" FLOAT, 
	"KPI_32" FLOAT, 
	"KPI_33" FLOAT, 
	"KPI_34" FLOAT, 
	"KPI_35" FLOAT, 
	"KPI_36" FLOAT, 


In [61]:
display(Markdown(output_challenge['input']))
display(Markdown(output_challenge['output']))

Which productline had the second average profit margin in the North region during 2025. And how much lower was the profit margin versus the highest average profit margin in the North region during 2025


The productline with the second highest average profit margin in the North region during 2025 was Furniture with an average profit margin of 0.325. The highest average profit margin was Electronics with 0.33, so Furniture's profit margin was 0.005 lower than the highest.

Explanation:  
I queried the `sales_db` table to find the average profit margin for each productline in the North region during the year 2025. I grouped the results by productline and ordered them by average profit margin in descending order. The query returned Electronics as the highest, Furniture as the second highest, and Clothing third. I then calculated the difference between the highest and second highest average profit margins.

The SQL query used was:

```sql
SELECT ProductLine, AVG(ProfitMargin) as AvgProfitMargin
FROM sales_db
WHERE Region = 'North' AND strftime('%Y', Month) = '2025'
GROUP BY ProductLine
ORDER BY AvgProfitMargin DESC
LIMIT 30
```